In [1]:
!pip install lyricsgenius
!pip install -U pip
!pip install -U dill
!pip install -U nltk

     |████████████████████████████████| 92kB 3.1MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
     |████████████████████████████████| 1.4MB 2.7MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
Requirement already up-to-date: dill in /usr/local/lib/python3.6/dist-packages (0.3.1.1)
     |████████████████████████████████| 1.5 MB 2.7 MB/s 
  Created wheel for nltk: filename=nltk-3.4.5-py3-none-any.whl size=1449905 sha256=78e8a39316a1a0e8fee2a8e8351e4ceeb9907a0bcebdfeba73b1f5251ed22fa7
  Stored in directory: /root/.cache/pip/wheels/e3/c9/b0/ed26a73ef75a53145820825afa8e2d2c9b30fe9f6c10cd3202
Successfully built nltk
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [2]:
import lyricsgenius
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
from nltk import word_tokenize
from nltk import sent_tokenize
from nltk import tokenize
from nltk.tokenize import LineTokenizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.model_selection import train_test_split
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten
from nltk.lm import MLE
from nltk.lm import Laplace
from nltk.lm import KneserNeyInterpolated
from nltk.lm.preprocessing import padded_everygram_pipeline
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
genius = lyricsgenius.Genius("iihP9u6cVKE79Xtu99OH6rVZlMUzIMeOvIVs8BDQNDeVmZhXJXEaI28E6QOc4BdB")
genius.remove_section_headers = True

In [0]:
artist = genius.search_artist("Oxxxymiron", sort="title")

In [5]:
# saving the lyrics of all songs of Oxxxymiron
artist.save_lyrics(extension='txt')

Wrote `Lyrics_Oxxxymiron.txt`


In [0]:
lyricss = open("Lyrics_Oxxxymiron.txt","r")

In [0]:
# tokenizing the lyrics, probably the tokenization is raw and there is anything I missed?
tokenized_text = [list(map(str.lower, word_tokenize(sent))) 
                  for sent in lyricss]

In [0]:
# Firstly, I was thinking, that adding \n symbol is a good idea to let the model know, where to start the next line,
# but then i neglected the idea
# for strokes in tokenized_text:
#   strokes.append('\n')

In [0]:
# def a function to generate rap text given a model
def rap_gen(model, num_lines = 20, num_words_in_line = 20, random_seed = 0):
  """
    :param model: An ngram language model from `nltk.lm.model`.
    :param num_words: Max no. of words to generate.
    :param random_seed: Seed value for random.
    """
  for i in range(num_lines):
    content = []
    for token in model.generate(num_words_in_line, random_seed=random_seed+i):
      if token == '<s>':
        continue
      if token == '</s>':
        break
      content.append(token)
    content = detokenize(content)
    if bool(content):
      i = 0
      while not content[i].isalpha():
          i += 1
          if i > len(content)-1:
            break
      print(content[i:].capitalize())



In [0]:
train, test = train_test_split(tokenized_text, test_size=0.3, random_state=11)

In [0]:
n = 3

In [0]:
train_data, padded_sents = padded_everygram_pipeline(n, train)

In [0]:
model = MLE(n)

In [0]:
model.fit(train_data, padded_sents)

In [95]:
rap_gen(model, random_seed = 10)

Декаданс
Priority, bubbly, i murder with intent
Антиквар пожмёт плечами: « дивиться нечему, делиться нечем »
И сам ты, давай, белые делают мясо похлеще
Экрана
К своей неуловимой ultima thule
Весь пирог, а подобострастный холоп, после - уроды и карлики
Лжи
То мой член уже весь в твоей ротовой полости
Что в 30 лет — финиш
У тени отца
Мы своровали одежду, (вау) и ты помнишь, я мир оплетаю, как молохом в пух перемолот
У тебя даже толстая кишка тонка
Небо, и не арлекин
Как режим пиночета
Высоток с парковкой для шестисотых


In [0]:
train_data2, padded_sents2 = padded_everygram_pipeline(n, train)

In [0]:
model2 = KneserNeyInterpolated(n)

In [0]:
model2.fit(train_data2, padded_sents2)

In [109]:
rap_gen(model2, random_seed = 10,num_lines=10,num_words_in_line=10)

Осталися в треклисте
Массы?
Мода - бред, имидж - ничто, но немыслимое
Ди па, сикиде пип
Белые кролики в головных уборах только в детройте
Чечёткой
Капитан у штурвала
Немнекеле быте
Вражий костолом
Почве могут открыться ворота


The generated texts are really strange, actually (however, i'm not a rap expert, probably, this is OK :)

Can you also give some advise on a model evaluation with perplexity. Thing I did below, unfortunately, do not give desired result.

In [0]:
test_data,_ = padded_everygram_pipeline(n, test)

In [0]:
test_gen = [list(line) for line in test_data]

In [138]:
model.perplexity(test_gen)

inf

In [139]:
model2.perplexity(test_gen)

ZeroDivisionError: ignored